<a href="https://colab.research.google.com/github/motasem00/data-driven-modelling/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import bibs
import os
import geopandas as gpd
from matplotlib.patches import Patch
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import TruncatedSVD
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

# Sklearn modules
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestRegressor
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans

In [5]:
%cd /content/

/content


In [ ]:
!ls

data-driven-modelling  sample_data


In [3]:
repo_dir = 'data-driven-modelling'
if os.path.exists(repo_dir):
    !rm -rf {repo_dir}

In [4]:
repo_url = 'https://github.com/motasem00/data-driven-modelling.git'
!git clone {repo_url}
%cd data-driven-modelling

Cloning into 'data-driven-modelling'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 106 (delta 52), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (106/106), 36.79 MiB | 5.58 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/content/data-driven-modelling


# Loading data

In [5]:
%cd /content/data-driven-modelling/notebooks//

/content/data-driven-modelling/notebooks


In [6]:
working_directory = os.getcwd()
path = working_directory
path

'/content/data-driven-modelling/notebooks'

In [7]:
df_gdp = pd.read_pickle("/content/data-driven-modelling/data/raw/gdp.pickle")
df_pattern = pd.read_pickle("/content/data-driven-modelling/data/raw/naics_pattern.pickle")
df_occupation_1 = pd.read_pickle("/content/data-driven-modelling/data/raw/naics_occupation_part1.pickle")
df_occupation_2 = pd.read_pickle("/content/data-driven-modelling/data/raw/naics_occupation_part2.pickle")
df_occupation_3 = pd.read_pickle("/content/data-driven-modelling/data/raw/naics_occupation_part3.pickle")

In [8]:
county_data = pd.read_pickle("/content/data-driven-modelling/data/raw/county.pickle")
State_data = pd.read_pickle("/content/data-driven-modelling/data/raw/state.pickle")

In [9]:
df_pattern['FIPS'] = df_pattern['FIPS'].astype(str)
unique_lengths = df_pattern['FIPS'].apply(len).unique()
unique_lengths

array([4, 5])

In [10]:
def add_zeros(code):
    if len(code) == 3:
        return '00' + code
    elif len(code) == 4:
        return '0' + code
    elif len(code) == 1:
        return '0000' + code
    return code

In [11]:
df_pattern['FIPS'] = df_pattern['FIPS'].apply(add_zeros)

In [12]:
df_occupation = pd.concat([df_occupation_1, df_occupation_2, df_occupation_3], ignore_index=True)
df_occupation.head(2)

,FIPS,State_GEOID,naics,NAICS_TITLE,emp_total_county_naics,OCC_CODE,OCC_TITLE,emp_occupation,state_name
0,12999,12,5613,Employment Services,1436559,49-9071,"Maintenance and Repair Workers, General",20639.514235,NaN
1,06999,6,5613,Employment Services,729335,49-9071,"Maintenance and Repair Workers, General",9414.167765,NaN


# My project plan will be as following:

* Upload and Link the datasets.
* And I will try to understand the data and find a way of matching things and find a working model.
* Then I will try to clean the data and drop the un-necessary columns.



### We will start with the GDP Data

In [13]:
df_gdp.columns

Index(['FIPS', 'GeoName', 'Region', 'TableName', 'LineCode',
       'IndustryClassification', 'Description', 'Unit', '2017', '2018', '2019',
       '2020', '2021', '2022'],
      dtype='object')

In [14]:
columns_to_drop_gdp = [
    'GeoName', 'Region', 'TableName', 'LineCode','Unit'
]

# Revomve not needed columns
filtered_gdp = df_gdp.drop(columns=columns_to_drop_gdp, errors='ignore')

# Show the rest
print(filtered_gdp.columns)

Index(['FIPS', 'IndustryClassification', 'Description', '2017', '2018', '2019',
       '2020', '2021', '2022'],
      dtype='object')


In [15]:
filtered_gdp

,FIPS,IndustryClassification,Description,2017,2018,2019,2020,2021,2022
0,0,...,All industry total,1.961210e+10,2.019390e+10,2.069209e+10,2.023407e+10,2.140769e+10,2.182204e+10
1,0,...,Private industries,1.715626e+10,1.771178e+10,1.819575e+10,1.776187e+10,1.890945e+10,1.928312e+10
2,0,11,"Agriculture, forestry, fishing and hunting",1.768400e+08,1.841050e+08,1.711490e+08,1.736590e+08,1.837420e+08,1.700580e+08
3,0,21,"Mining, quarrying, and oil and gas extraction",2.673020e+08,2.770130e+08,3.146040e+08,3.055880e+08,2.694780e+08,2.452900e+08
4,0,22,Utilities,3.137110e+08,3.092690e+08,3.129200e+08,3.323360e+08,3.166030e+08,3.183920e+08
...,...,...,...,...,...,...,...,...,...
108047,98000,"42,44-45",Trade,4.461445e+08,4.576763e+08,4.741341e+08,4.674147e+08,4.661462e+08,4.292295e+08
108048,98000,"22,48-49",Transportation and utilities,1.674685e+08,1.733558e+08,1.814022e+08,1.711564e+08,1.857088e+08,1.903768e+08
108049,98000,"31-33,51",Manufacturing and information,7.469147e+08,7.976669e+08,8.595105e+08,8.771524e+08,9.864447e+08,1.010181e+09
108050,98000,...,Private goods-producing industries 2/,6.350413e+08,6.706487e+08,6.898229e+08,6.691179e+08,6.954938e+08,6.650949e+08


In [16]:
# Convert GDP_2022 column to numeric
filtered_gdp['2022'] = pd.to_numeric(filtered_gdp['2022'], errors='coerce')

In [17]:
columns_to_drop_gdp = [
    'IndustryClassification',  '2017', '2018', '2019', '2020', '2021'
]

# Revomve not needed columns
df_filtered_gdp = filtered_gdp.drop(columns=columns_to_drop_gdp, errors='ignore')

# Show the rest
print(df_filtered_gdp.columns)

Index(['FIPS', 'Description', '2022'], dtype='object')


In [18]:
df_filtered_gdp['2022'] = pd.to_numeric(df_filtered_gdp['2022'], errors='coerce')

In [19]:
df_filtered_gdp.dtypes

,0
FIPS,int64
Description,object
2022,float64


In [20]:
df_filtered_gdp['2022'].fillna(0, inplace=True)

<ipython-input-20-014d08453687>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filtered_gdp['2022'].fillna(0, inplace=True)


In [21]:
df_filtered_gdp.head()

,FIPS,Description,2022
0,0,All industry total,2.182204e+10
1,0,Private industries,1.928312e+10
2,0,"Agriculture, forestry, fishing and hunting",1.700580e+08
3,0,"Mining, quarrying, and oil and gas extraction",2.452900e+08
4,0,Utilities,3.183920e+08


### Then I Will have a look at the pattern dataset

In [22]:
columns_to_drop_pattern = [
    'State_GEOID', 'County_GEOID','naics_2', 'emp_nf', 'qp1_nf', 'qp1', 'ap_nf',
    'n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499',
       'n500_999', 'n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4'
]

# Revomve not needed columns
filtered_pattern = df_pattern.drop(columns=columns_to_drop_pattern, errors='ignore')

# Show the rest
print(filtered_pattern.columns)

Index(['FIPS', 'naics', 'DESCRIPTION', 'emp', 'ap', 'est'], dtype='object')


In [23]:
filtered_pattern = filtered_pattern.astype({"FIPS": int})

In [24]:
columns_to_drop_occupation = [
    'State_GEOID',  'emp_total_county_naics', 'state_name'
]

# Revomve not needed columns
df_filtered_occupation_main = df_occupation.drop(columns=columns_to_drop_occupation, errors='ignore')

# Show the rest
print(df_filtered_occupation_main.columns)

Index(['FIPS', 'naics', 'NAICS_TITLE', 'OCC_CODE', 'OCC_TITLE',
       'emp_occupation'],
      dtype='object')


In [25]:
df_filtered_occupation_main.rename(columns={'GEOID': 'FIPS'}, inplace=True)

In [26]:
df_filtered_occupation_main = df_filtered_occupation_main.astype({"FIPS": int})

### Let's now have a look at the Occupation Dataset

In [27]:
columns_to_drop_occupation = [
    'State_GEOID',  'emp_total_county_naics', 'state_name'
]

# Revomve not needed columns
df_filtered_occupation = df_occupation.drop(columns=columns_to_drop_occupation, errors='ignore')

# Show the rest
print(df_filtered_occupation.columns)

Index(['FIPS', 'naics', 'NAICS_TITLE', 'OCC_CODE', 'OCC_TITLE',
       'emp_occupation'],
      dtype='object')


In [28]:
df_filtered_occupation.rename(columns={'GEOID': 'FIPS'}, inplace=True)

In [29]:
df_filtered_occupation = df_filtered_occupation.astype({"FIPS": int})

### We Got a problem

---


*When we did the ranking for industries we found some NAICS are in 6 digits and that's why we had to create a function to expand those NAICS to it's representative 4 digit Industries codes*

In [30]:
pattern_data = filtered_pattern

In [31]:
print(pattern_data.columns)

Index(['FIPS', 'naics', 'DESCRIPTION', 'emp', 'ap', 'est'], dtype='object')


In [32]:
  naics_expanding = {
        '3330A1': {
            'codes': ['3331', '3332', '3334', '3339'],
            'description': [
                'Agriculture, Construction, and Mining Machinery Manufacturing',
                'Industrial Machinery Manufacturing',
                'Ventilation, Heating, Air-Conditioning, and Commercial Refrigeration Equipment Manufacturing',
                'Other General Purpose Machinery Manufacturing'
            ]
        },

         '3330A1': {
            'codes': ['3323', '3324'],
            'description': [
                'Architectural and Structural Metals Manufacturing',
                'Boiler, Tank, and Shipping Container Manufacturing',
            ]
        },

        '3320A1': {
            'codes': ['3321', '3322', '3325', '3326', '3329'],
            'description': [
                'Forging and Stamping',
                'Cutlery and Handtool Manufacturing',
                'Hardware Manufacturing',
                'Spring and Wire Product Manufacturing',
                'Other Fabricated Metal Product Manufacturing'
            ]
        },

        '3250A1': {
            'codes': ['3251', '3252', '3253', '3259'],
            'description': [
                'Basic Chemical Manufacturing',
                'Resin, Synthetic Rubber, and Artificial and Synthetic Fibers and Filaments Manufacturing',
                'Pesticide, Fertilizer, and Other Agricultural Chemical Manufacturing',
                'Other Chemical Product and Preparation Manufacturing'
            ]
        },

        '3370A1': {
            'codes': ['3371', '3372'],
            'description': [
                'Household and Institutional Furniture and Kitchen Cabinet Manufacturing',
                'Office Furniture (including Fixtures) Manufacturing'
            ]
        },
    }

In [33]:
import sys
sys.path.append('/content/data-driven-modelling/notebooks')

from func import DataWrangling

In [34]:
wrangler = DataWrangling()

In [35]:
df_pattern = pattern_data

In [36]:
processed_df = wrangler.process_naics_data(df_pattern, naics_expanding)

In [37]:
processed_df.head()

,FIPS,naics,DESCRIPTION,emp,ap,est
0,1001,1133,Logging,68,4563,7
1,1001,2123,Nonmetallic Mineral Mining and Quarrying,87,5144,6
2,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4
3,1001,2362,Nonresidential Building Construction,69,4184,7
4,1001,2371,Utility System Construction,65,4574,4


In [38]:
# # NAICS mapping

# naics_mapping = {
#     "3111": "Animal Food Manufacturing",
#     "3112": "Grain and Oilseed Milling",
#     "3113": "Sugar and Confectionery Product Manufacturing",
#     "3114": "Fruit and Vegetable Preserving and Specialty Food Manufacturing",
#     "3115": "Dairy Product Manufacturing",
#     "3116": "Animal Slaughtering and Processing",
#     "3117": "Seafood Product Preparation and Packaging",
#     "3118": "Bakeries and Tortilla Manufacturing",
#     "3119": "Other Food Manufacturing",
#     "3121": "Beverage Manufacturing",
#     "3122": "Tobacco Manufacturing",
#     "3131": "Fiber, Yarn, and Thread Mills",
#     "3132": "Fabric Mills",
#     "3133": "Textile and Fabric Finishing and Fabric Coating Mills",
#     "3141": "Textile Product Mills (except Carpet and Rug)",
#     "3149": "Carpet and Rug Mills",
#     "3151": "Apparel Knitting Mills",
#     "3152": "Cut and Sew Apparel Manufacturing",
#     "3159": "Apparel Accessories and Other Apparel Manufacturing",
#     "3161": "Leather and Hide Tanning and Finishing",
#     "3162": "Footwear Manufacturing",
#     "3169": "Other Leather and Allied Product Manufacturing",
#     "3211": "Sawmills and Wood Preservation",
#     "3212": "Veneer, Plywood, and Engineered Wood Product Manufacturing",
#     "3219": "Other Wood Product Manufacturing",
#     "3221": "Pulp, Paper, and Paperboard Mills",
#     "3222": "Converted Paper Product Manufacturing",
#     "3231": "Printing and Related Support Activities",
#     "3241": "Petroleum and Coal Products Manufacturing",
#     "3251": "Basic Chemical Manufacturing",
#     "3252": "Resin, Synthetic Rubber, and Artificial Synthetic Fibers and Filaments Manufacturing",
#     "3253": "Pesticide, Fertilizer, and Other Agricultural Chemical Manufacturing",
#     "3254": "Pharmaceutical and Medicine Manufacturing",
#     "3255": "Paint, Coating, and Adhesive Manufacturing",
#     "3256": "Soap, Cleaning Compound, and Toilet Preparation Manufacturing",
#     "3259": "Other Chemical Product and Preparation Manufacturing",
#     "3261": "Plastics Product Manufacturing",
#     "3262": "Rubber Product Manufacturing",
#     "3271": "Clay Product and Refractory Manufacturing",
#     "3272": "Glass and Glass Product Manufacturing",
#     "3273": "Cement and Concrete Product Manufacturing",
#     "3274": "Lime and Gypsum Product Manufacturing",
#     "3279": "Other Nonmetallic Mineral Product Manufacturing",
#     "3311": "Iron and Steel Mills and Ferroalloy Manufacturing",
#     "3312": "Steel Product Manufacturing from Purchased Steel",
#     "3313": "Alumina and Aluminum Production and Processing",
#     "3314": "Nonferrous Metal (except Aluminum) Production and Processing",
#     "3315": "Foundries",
#     "3321": "Forging and Stamping",
#     "3322": "Cutlery and Handtool Manufacturing",
#     "3323": "Architectural and Structural Metals Manufacturing",
#     "3324": "Boiler, Tank, and Shipping Container Manufacturing",
#     "3325": "Hardware Manufacturing",
#     "3326": "Spring and Wire Product Manufacturing",
#     "3327": "Machine Shops",
#     "3328": "Coating, Engraving, Heat Treating, and Allied Activities",
#     "3329": "Other Fabricated Metal Product Manufacturing",
#     "3331": "Agriculture, Construction, and Mining Machinery Manufacturing",
#     "3332": "Industrial Machinery Manufacturing",
#     "3333": "Commercial and Service Industry Machinery Manufacturing",
#     "3334": "Ventilation, Heating, Air-Conditioning, and Commercial Refrigeration Equipment Manufacturing",
#     "3335": "Metalworking Machinery Manufacturing",
#     "3336": "Engine, Turbine, and Power Transmission Equipment Manufacturing",
#     "3339": "Other General Purpose Machinery Manufacturing",
#     "3341": "Computer and Peripheral Equipment Manufacturing",
#     "3342": "Communications Equipment Manufacturing",
#     "3343": "Audio and Video Equipment Manufacturing",
#     "3344": "Semiconductor and Other Electronic Component Manufacturing",
#     "3345": "Navigational, Measuring, Electromedical, and Control Instruments Manufacturing",
#     "3346": "Manufacturing and Reproducing Magnetic and Optical Media",
#     "3351": "Electric Lighting Equipment Manufacturing",
#     "3352": "Household Appliance Manufacturing",
#     "3353": "Electrical Equipment Manufacturing",
#     "3359": "Other Electrical Equipment and Component Manufacturing",
#     "3361": "Motor Vehicle Manufacturing",
#     "3362": "Motor Vehicle Body and Trailer Manufacturing",
#     "3363": "Motor Vehicle Parts Manufacturing",
#     "3364": "Aerospace Product and Parts Manufacturing",
#     "3365": "Railroad Rolling Stock Manufacturing",
#     "3366": "Ship and Boat Building",
#     "3369": "Other Transportation Equipment Manufacturing",
#     "3371": "Household and Institutional Furniture and Kitchen Cabinet Manufacturing",
#     "3372": "Office Furniture (including Fixtures) Manufacturing",
#     "3379": "Other Furniture Related Product Manufacturing",
#     "3391": "Medical Equipment and Supplies Manufacturing",
#     "3399": "Other Miscellaneous Manufacturing",
# }
# # Function to map descriptions
# def map_description(description, naics_mapping):
#     # Split the description into individual codes
#     naics_codes = description.split(", ")
#     # Map each code to its industry using the dictionary
#     mapped_industries = [naics_mapping.get(code, code) for code in naics_codes]
#     # Join the mapped industries back together
#     return ", ".join(mapped_industries)

# # Apply the function to the DESCRIPTION column
# processed_df['DESCRIPTION'] = processed_df['DESCRIPTION'].apply(lambda desc: map_description(desc, naics_mapping))

# # Display the updated DataFrame
# processed_df.head(5)

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

# 1. Data Preparation

def assign_tool_consumption(description):
    """
    Assigns tool consumption level based on industry descriptions.
    """
    description = description.lower()

    # Highest tool consumption (level 9)
    if any(keyword in description for keyword in [
        "metalworking machinery",
        "structural metals manufacturing",
        "machine shops",
        "forging",
        "iron and steel mills",
        "steel product manufacturing",
        "aluminum production",
        "nonferrous metal production"
    ]):
        return 9

    # Very high tool consumption (level 8)
    if any(keyword in description for keyword in [
        "motor vehicle manufacturing",
        "motor vehicle parts manufacturing",
        "ship and boat building",
        "automotive repair",
        "automotive equipment rental"
    ]):
        return 8

    # High tool consumption (level 7)
    if any(keyword in description for keyword in [
        "construction machinery",
        "heavy and civil engineering construction",
        "industrial equipment repair",
        "coating and engraving"
    ]):
        return 7

    # Moderate tool consumption (level 5-6)
    if any(keyword in description for keyword in [
        "chemical manufacturing",
        "rubber product manufacturing",
        "nonmetallic mineral mining",
        "petroleum product manufacturing",
        "building equipment contractors"
    ]):
        return np.random.randint(5, 7)  # Assign a random level between 5 and 6

    # Low tool consumption (level 3-4)
    if any(keyword in description for keyword in [
        "woodworking",
        "paper product manufacturing",
        "sawmills"
    ]):
        return np.random.randint(3, 5)  # Assign a random level between 3 and 4

    # Very low tool consumption (level 1-2)
    if any(keyword in description for keyword in [
        "utilities",
        "energy distribution",
        "support activities",
        "employment services"
    ]):
        return np.random.randint(1, 3)  # Assign a random level between 1 and 2

    # Default level if no keywords match
    return 0

# Apply the function to create the 'Tool Consumption 1st' column
processed_df['Tool Consumption 1st'] = processed_df['DESCRIPTION'].apply(assign_tool_consumption)

# Preprocess the text descriptions
def preprocess_text(text):
    text = text.lower()
    return text

processed_df['DESCRIPTION'] = processed_df['DESCRIPTION'].apply(preprocess_text)

# Split data into training and validation sets
X = processed_df['DESCRIPTION']
y = processed_df['Tool Consumption 1st']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Reset the index of the split datasets
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

# 2. Feature Engineering
vectorizer = TfidfVectorizer(max_features=2000)  # Adjust max_features as needed
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

# 3. Model Training (using Logistic Regression)
model = LogisticRegression(max_iter=5000)  # Adjust max_iter as needed
model.fit(X_train_vec, y_train)

# 4. Prediction and Rule-Based Adjustment
predictions = model.predict(X_val_vec)

# Enhanced adjust_with_rules function with more specific rules
def adjust_with_rules(row, prediction):
    """
    Adjusts the prediction based on rule-based logic.
    """
    description = row.DESCRIPTION.lower()  # Access DESCRIPTION as an attribute of the row tuple

    # Highest priority adjustments (override model prediction)
    if "metalworking" in description:
        return 9
    if "automotive repair" in description:
        return 8

    return prediction  # Keep the original prediction if no rules match

# Predict on the entire dataset
X_all_vec = vectorizer.transform(processed_df['DESCRIPTION'])  # Transform all descriptions
predictions_all = model.predict(X_all_vec)

# Apply rule-based adjustment to all predictions
processed_df['Tool Consumption'] = [
    adjust_with_rules(row, pred) for row, pred in zip(processed_df.itertuples(index=False), predictions_all)
]

# Remove records with Tool Consumption of 0 or 1
final_df = processed_df[~processed_df['Tool Consumption'].isin([0, 1])]

# Print the updated DataFrame with all original columns plus \\'Tool Consumption'
print(final_df.head(10))

# Optional: Evaluate the model
print(classification_report(y_val, predictions))

    FIPS naics                                     DESCRIPTION   emp     ap  \
1   1001  2123        nonmetallic mineral mining and quarrying    87   5144   
6   1001  2382                  building equipment contractors   157   7822   
8   1001  3231         printing and related support activities    33   1334   
15  1001  8111               automotive repair and maintenance    93   3894   
16  1003  2123        nonmetallic mineral mining and quarrying    29   1291   
17  1003  2131                   support activities for mining     6    342   
22  1003  2379  other heavy and civil engineering construction    74   3867   
24  1003  2382                  building equipment contractors  1126  54263   
28  1003  3222           converted paper product manufacturing   241  14546   
29  1003  3231         printing and related support activities    92   2820   

    est  Tool Consumption  Tool Consumption 1st  
1     6                 6                     5  
6    27                 5     

In [54]:
final_df = final_df.drop('Tool Consumption 1st', axis=1)

In [55]:
final_df.head(5)

,FIPS,naics,DESCRIPTION,emp,ap,est,Tool Consumption
1,1001,2123,nonmetallic mineral mining and quarrying,87,5144,6,6
6,1001,2382,building equipment contractors,157,7822,27,5
8,1001,3231,printing and related support activities,33,1334,4,2
15,1001,8111,automotive repair and maintenance,93,3894,20,8
16,1003,2123,nonmetallic mineral mining and quarrying,29,1291,3,6


In [56]:
pre_industry_feature = final_df.groupby(['naics', 'DESCRIPTION', 'Tool Consumption']).agg(
    total_emp=('emp', 'sum'),            # Sum of values in the 'emp'
    total_est=('est', 'sum'),
    unique_FIPS=('FIPS', 'nunique')    # Count of unique values in the 'FIPS' column
).sort_values(by='total_emp', ascending=False).reset_index()

# Display the first 20 rows
pre_industry_feature.head(5)

,naics,DESCRIPTION,Tool Consumption,total_emp,total_est,unique_FIPS
0,2382,building equipment contractors,5,2171001,193562,2756
1,8111,automotive repair and maintenance,8,902870,164584,2725
2,3363,motor vehicle parts manufacturing,8,464689,3700,439
3,3231,printing and related support activities,2,356146,21376,1116
4,3327,"machine shops; turned product; and screw, nut,...",9,332653,20979,1276


In [57]:
# Create a copy of the DataFrame
pre_feature_result_copy = pre_industry_feature.copy()

# Calculate rank for all columns starting from 'tool_consumption', with equal values having the same rank
for column in pre_feature_result_copy.columns[pre_feature_result_copy.columns.get_loc('Tool Consumption'):]:
    rank_column_name = f'rank_{column}'
    pre_feature_result_copy[rank_column_name] = pre_feature_result_copy[column].rank(method='min', ascending=False).astype(int)

# Display the first few rows of the copied data
pre_feature_result_copy.head()

,naics,DESCRIPTION,Tool Consumption,total_emp,total_est,unique_FIPS,rank_Tool Consumption,rank_total_emp,rank_total_est,rank_unique_FIPS
0,2382,building equipment contractors,5,2171001,193562,2756,14,1,1,1
1,8111,automotive repair and maintenance,8,902870,164584,2725,6,2,2,2
2,3363,motor vehicle parts manufacturing,8,464689,3700,439,6,3,9,11
3,3231,printing and related support activities,2,356146,21376,1116,17,4,3,4
4,3327,"machine shops; turned product; and screw, nut,...",9,332653,20979,1276,1,5,4,3


In [62]:
weights = {
    'rank_Tool Consumption': 0.5,   # Weight for the ranking column 'tool_consumption'
    'rank_total_emp': 0.3,            # Weight for the ranking column 'emp_sum'
    'rank_total_est': 0.2,         # Weight for the ranking column 'unique_FIPS'
}

In [63]:
# calculate the weighted sum
pre_feature_result_copy['Weighted_Sum'] = (pre_feature_result_copy['rank_Tool Consumption'] * weights['rank_Tool Consumption'] +
                          pre_feature_result_copy['rank_total_emp'] * weights['rank_total_emp'] +
                          pre_feature_result_copy['rank_total_est'] * weights['rank_total_est']
                          ) # Display the first few rows
pre_feature_result_sorted = pre_feature_result_copy.sort_values(by='Weighted_Sum', ascending=True)

In [64]:
pre_feature_result_sorted.head(10)

,naics,DESCRIPTION,Tool Consumption,total_emp,total_est,unique_FIPS,rank_Tool Consumption,rank_total_emp,rank_total_est,rank_unique_FIPS,Weighted_Sum
4,3327,"machine shops; turned product; and screw, nut,...",9,332653,20979,1276,1,5,4,3,2.8
1,8111,automotive repair and maintenance,8,902870,164584,2725,6,2,2,2,4.0
9,3335,metalworking machinery manufacturing,9,117724,5141,455,1,10,8,10,5.1
2,3363,motor vehicle parts manufacturing,8,464689,3700,439,6,3,9,11,5.7
8,5321,automotive equipment rental and leasing,8,137245,14677,784,6,9,5,5,6.7
0,2382,building equipment contractors,5,2171001,193562,2756,14,1,1,1,7.5
16,3311,iron and steel mills and ferroalloy manufacturing,9,44071,152,34,1,17,17,18,9.0
17,3312,steel product manufacturing from purchased steel,9,21490,282,54,1,18,16,16,9.1
10,3366,ship and boat building,8,110876,968,126,6,11,15,15,9.3
18,3313,alumina and aluminum production and processing,9,20149,147,36,1,19,18,17,9.8


In [65]:
top5_industries = pre_feature_result_sorted[:5]['naics'].tolist()
top5_industries

['3327', '8111', '3335', '3363', '5321']

In [99]:
naics_top6 = ['2382','8111','3320A2','3330A1','3327','3363']
naics_top_metall = ['3320A1','3335','3364','3362','3315','3366','2379','3336','3311','3314']

In [43]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
# import torch
# import numpy as np  # Make sure to import numpy


# def assign_tool_consumption(description):
#     """
#     Assigns tool consumption level based on AI-powered analysis of industry descriptions.
#     """
#     description = description.lower()

#     # High tool consumption industries (level 8-9)
#     if any(keyword in description for keyword in [
#         "metal fabrication", "metal stamping", "machine shops", "metalworking machinery",
#         "sawmills", "woodworking machinery", "construction machinery", "mining machinery",
#         "agricultural machinery", "industrial machinery", "heavy equipment", "power tools",
#         "hand tools", "cutting tools", "welding", "forging", "casting", "molding"
#     ]):
#         return np.random.randint(8, 10)  # Assign a random level between 8 and 9

#     # Moderate tool consumption industries (level 5-7)
#     if any(keyword in description for keyword in [
#         "manufacturing", "construction", "repair", "maintenance", "assembly", "fabrication",
#         "aerospace", "automotive", "electronics", "furniture", "plastics", "rubber",
#         "chemicals", "textiles", "printing", "publishing"
#     ]):
#         return np.random.randint(5, 8)  # Assign a random level between 5 and 7

#     # Low tool consumption industries (level 1-4)
#     if any(keyword in description for keyword in [
#         "agriculture", "forestry", "fishing", "mining", "utilities", "wholesale trade",
#         "retail trade", "transportation", "warehousing", "information", "finance",
#         "insurance", "real estate", "professional services", "management",
#         "administrative", "education", "healthcare", "arts", "entertainment",
#         "food services", "accommodation", "other services"
#     ]):
#         return np.random.randint(1, 5)  # Assign a random level between 1 and 4

#     # Default level if no keywords match
#     return 1

# # Apply the function to create the 'Tool Consumption (AI)' column
# processed_df['Tool Consumption'] = processed_df['DESCRIPTION'].apply(assign_tool_consumption)


# # Preprocess the text descriptions
# def preprocess_text(text):
#     text = text.lower()
#     # Add more preprocessing steps if needed (e.g., remove punctuation, stemming)
#     return text

# processed_df['DESCRIPTION'] = processed_df['DESCRIPTION'].apply(preprocess_text)

# # Split data into training and validation sets
# X = processed_df['DESCRIPTION']
# y = processed_df['Tool Consumption']
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Reset the index of the split datasets (important to avoid KeyError)
# X_train = X_train.reset_index(drop=True)
# X_val = X_val.reset_index(drop=True)
# y_train = y_train.reset_index(drop=True)
# y_val = y_val.reset_index(drop=True)

# # 2. Model Training (using BERT)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=10)

# class MyDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# # Prepare data for the trainer
# train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
# val_encodings = tokenizer(list(X_val), truncation=True, padding=True)
# train_dataset = MyDataset(train_encodings, y_train)
# val_dataset = MyDataset(val_encodings, y_val)

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=1,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=64,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10,
#     report_to="none",  # Add this line to disable wandb
#     optim="adamw_torch",
#     gradient_accumulation_steps=2
# )

# # Create Trainer instance
# trainer = Trainer(  # This line defines the 'trainer' variable
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset
# )

# # Train the model
# trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,2.180400
20,2.148700
30,2.120600
40,2.010400
50,1.902900
60,1.848400
70,1.720000
80,1.661200
90,1.495800


KeyboardInterrupt: 

In [ ]:
# from main import DataWrangling  # Assuming the class is in a file named main.py

# # Step 1: Load data
# # gdp_data = filtered_gdp
# # occupation_data = filtered_occupation

# # # Step 2: Expand NAICS codes
# pattern_data_expanded = DataWrangling.expand_naics_details(pattern_data, naics_expanding)

# # # Step 3: Merge GDP and Pattern data
# # merged_data = DataWrangling.merge_and_filter_datasets(gdp_data, pattern_data_expanded)

# # # Step 4: Aggregate data
# # aggregated_data = DataProcessing.aggregate_data(merged_data)

# # # Step 5: Process Occupation data
# # filtered_occ_codes = ['47-1011', '49-1011']  # Replace with your prioritized codes
# # processed_occupation_data = DataProcessing.process_occupation_data(occupation_data, filtered_occ_codes)

# # # Step 6: Combine datasets
# # final_dataset = DataProcessing.final_merge_and_aggregation(aggregated_data, processed_occupation_data)

# # # Step 7: Save final dataset
# # final_dataset.to_pickle('processed_data/pkl/final_dataset.pkl')



,FIPS,naics,DESCRIPTION,emp,ap,est
0,1001,1133,Logging,68,4563,7
1,1001,2123,Nonmetallic Mineral Mining and Quarrying,87,5144,6
2,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4
3,1001,2362,Nonresidential Building Construction,69,4184,7
4,1001,2371,Utility System Construction,65,4574,4


# Tool consumption

Here, the individual industries must be evaluated by **domain knowledge** with regard to their potential tool consumption.<br>
We will insert random values between 0 and 10 as an example

**We gonna do that 2 times**

*   With the Industry Pattern Dataset
*   With the Occupation Dataset

<br>

---




**1.   FIRST Start with the Industries Pattern **




In [ ]:
# Generate random values for each NAICS and assign them to each entry in the group
random_values = {naics: np.random.randint(0, 11) for naics in processed_df['naics'].unique()}
processed_df['tool_consumption'] = processed_df['naics'].map(random_values)
processed_df.head(3)

,FIPS,naics,DESCRIPTION,emp,ap,est,tool_consumption
0,1001,1133,Logging,68,4563,7,6
1,1001,2123,Nonmetallic Mineral Mining and Quarrying,87,5144,6,4
2,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,7


In [ ]:
# Generate random values for each NAICS and assign them to each entry in the group
random_values = {naics: np.random.randint(0, 11) for naics in df_filtered_occupation['naics'].unique()}
df_filtered_occupation['tool_consumption_occ'] = df_filtered_occupation['naics'].map(random_values)
df_filtered_occupation.head(3)

,FIPS,naics,NAICS_TITLE,OCC_CODE,OCC_TITLE,emp_occupation,tool_consumption_occ
0,12999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",20639.514235,6
1,6999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",9414.167765,6
2,36999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",8332.850279,6


## Industries Before Ranking

In [ ]:
pattern_result = processed_df.groupby(['naics', 'DESCRIPTION', 'tool_consumption']).agg(
    emp_sum=('emp', 'sum'),            # Sum of values in the 'emp' column
    unique_FIPS=('FIPS', 'nunique')    # Count of unique values in the 'FIPS' column
).sort_values(by='emp_sum', ascending=False).reset_index()

# Display the first 20 rows
pattern_result.head(20)

,naics,DESCRIPTION,tool_consumption,emp_sum,unique_FIPS
0,5613,Employment Services,5,7061425,1306
1,2382,Building Equipment Contractors,6,2171001,2756
2,5617,Services to Buildings and Dwellings,7,2004960,2530
3,5413,"Architectural, Engineering, and Related Services",6,1579218,1924
4,4411,Automobile Dealers,9,1245435,1953
5,7139,Other Amusement and Recreation Industries,1,1092894,2182
6,8111,Automotive Repair and Maintenance,0,902870,2725
7,2381,"Foundation, Structure, and Building Exterior C...",2,901696,2313
8,2383,Building Finishing Contractors,4,811812,2040
9,4238,"Machinery, Equipment, and Supplies Merchant Wh...",1,739134,1947


In [ ]:
# Create a copy of the DataFrame
pattern_result_copy = pattern_result.copy()

# Calculate rank for all columns starting from 'tool_consumption', with equal values having the same rank
for column in pattern_result_copy.columns[pattern_result_copy.columns.get_loc('tool_consumption'):]:
    rank_column_name = f'rank_{column}'
    pattern_result_copy[rank_column_name] = pattern_result_copy[column].rank(method='min', ascending=False).astype(int)

# Display the first few rows of the copied data
pattern_result_copy.head()

,naics,DESCRIPTION,tool_consumption,emp_sum,unique_FIPS,rank_tool_consumption,rank_emp_sum,rank_unique_FIPS
0,5613,Employment Services,5,7061425,1306,30,1,14
1,2382,Building Equipment Contractors,6,2171001,2756,23,2,1
2,5617,Services to Buildings and Dwellings,7,2004960,2530,17,3,3
3,5413,"Architectural, Engineering, and Related Services",6,1579218,1924,23,4,10
4,4411,Automobile Dealers,9,1245435,1953,10,5,8


## Industries After Ranking

Adding Weights and for sure the highest weight will go for the **Rank Tool Consumption**

In [ ]:
weights = {
    'rank_tool_consumption': 0.5,   # Weight for the ranking column 'tool_consumption'
    'rank_emp_sum': 0.3,            # Weight for the ranking column 'emp_sum'
    'rank_unique_FIPS': 0.2,         # Weight for the ranking column 'unique_FIPS'
}

In [ ]:
# calculate the weighted sum
pattern_result_copy['Weighted_Sum'] = (pattern_result_copy['rank_tool_consumption'] * weights['rank_tool_consumption'] +
                          pattern_result_copy['rank_emp_sum'] * weights['rank_emp_sum'] +
                          pattern_result_copy['rank_unique_FIPS'] * weights['rank_unique_FIPS']
                          ) # Display the first few rows
pattern_sorted = pattern_result_copy.sort_values(by='Weighted_Sum', ascending=True)

In [ ]:
#Final Sorted Top Industries to be used later for filtering industies
pattern_sorted.head(20)

,naics,DESCRIPTION,tool_consumption,emp_sum,unique_FIPS,rank_tool_consumption,rank_emp_sum,rank_unique_FIPS,Weighted_Sum
13,4413,"Automotive Parts, Accessories, and Tire Stores",10,532828,2427,1,14,4,5.5
4,4411,Automobile Dealers,9,1245435,1953,10,5,8,8.1
2,5617,Services to Buildings and Dwellings,7,2004960,2530,17,3,3,10.0
22,3320,"3323, 3323",10,343891,1057,1,23,19,11.2
1,2382,Building Equipment Contractors,6,2171001,2756,23,2,1,12.3
25,2373,"Highway, Street, and Bridge Construction",10,277718,905,1,26,22,12.7
24,3320,"3321, 3322, 3325, 3326, 3329",10,315026,592,1,25,32,14.4
3,5413,"Architectural, Engineering, and Related Services",6,1579218,1924,23,4,10,14.7
16,2211,"Electric Power Generation, Transmission and Di...",7,451638,1274,17,17,16,16.8
11,3261,Plastics Product Manufacturing,7,572426,763,17,12,27,17.5


In [ ]:
top20_industries = pattern_sorted[:20]['naics'].tolist()
top20_industries

['4413',
 '4411',
 '5617',
 '3320',
 '2382',
 '2373',
 '3320',
 '5413',
 '2211',
 '3261',
 '5613',
 '8113',
 '3370',
 '3391',
 '3344',
 '3328',
 '2379',
 '5612',
 '3222',
 '2383']

**The following industries needs to be cleaned**

*   3320A1
*   3330A1

Will need to use the link here https://www.bls.gov/oes/2023/may/naics4_3320A2.htm <br> to Check what those 6 dig codes represesnts actually in 4 gid







---





**2.   SECOND Move to the Occupation **




In [ ]:
df_filtered_occupation.head()


,FIPS,naics,NAICS_TITLE,OCC_CODE,OCC_TITLE,emp_occupation,tool_consumption_occ
0,12999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",20639.514235,6
1,6999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",9414.167765,6
2,36999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",8332.850279,6
3,6037,8111,Automotive Repair and Maintenance,49-3023,Automotive Service Technicians and Mechanics,5913.423292,0
4,48999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",5770.378034,6


In [ ]:
occupation_result = df_filtered_occupation.groupby(['OCC_CODE','OCC_TITLE', 'tool_consumption_occ']).agg(
    emp_occ_sum=('emp_occupation', 'sum'),             # Sum of values in the 'emp' column
    unique_FIPS=('FIPS', 'nunique'),    # Count of unique values in the 'FIPS' column
).sort_values(by='emp_occ_sum', ascending=False).reset_index()

# Display the first 20 rows
occupation_result.head(20)

,OCC_CODE,OCC_TITLE,tool_consumption_occ,emp_occ_sum,unique_FIPS
0,49-3023,Automotive Service Technicians and Mechanics,0,251847.475177,2764
1,49-9071,"Maintenance and Repair Workers, General",6,158901.751652,2549
2,51-4041,Machinists,7,120619.327200,1570
3,51-4072,"Molding, Coremaking, and Casting Machine Sette...",0,104459.866587,776
4,49-3021,Automotive Body and Related Repairers,0,103113.463321,2725
5,17-2141,Mechanical Engineers,8,89165.227078,1923
6,49-9041,Industrial Machinery Mechanics,1,76794.846772,2090
7,51-4121,"Welders, Cutters, Solderers, and Brazers",4,72432.436290,1350
8,51-4121,"Welders, Cutters, Solderers, and Brazers",7,71863.370712,1554
9,17-2112,Industrial Engineers,8,64679.936227,1982


In [ ]:
# Create a copy of the DataFrame
occupation_result_copy = occupation_result.copy()

# Calculate rank for all columns starting from 'tool_consumption', with equal values having the same rank
for column in occupation_result_copy.columns[occupation_result_copy.columns.get_loc('tool_consumption_occ'):]:
    rank_column_name = f'rank_{column}'
    occupation_result_copy[rank_column_name] = occupation_result_copy[column].rank(method='min', ascending=False).astype(int)

# Display the first few rows of the copied data
occupation_result_copy.head()

,OCC_CODE,OCC_TITLE,tool_consumption_occ,emp_occ_sum,unique_FIPS,rank_tool_consumption_occ,rank_emp_occ_sum,rank_unique_FIPS
0,49-3023,Automotive Service Technicians and Mechanics,0,251847.475177,2764,570,1,9
1,49-9071,"Maintenance and Repair Workers, General",6,158901.751652,2549,217,2,20
2,51-4041,Machinists,7,120619.327200,1570,159,3,137
3,51-4072,"Molding, Coremaking, and Casting Machine Sette...",0,104459.866587,776,570,4,413
4,49-3021,Automotive Body and Related Repairers,0,103113.463321,2725,570,5,14


In [ ]:
weights = {
    'rank_tool_consumption_occ': 0.5,   # Weight for the ranking column 'tool_consumption'
    'rank_emp_occ_sum': 0.3,            # Weight for the ranking column 'emp_sum'
    'rank_unique_FIPS': 0.2,         # Weight for the ranking column 'unique_FIPS'
}


In [ ]:
# calculate the weighted sum
occupation_result_copy['Weighted_Sum_Occ'] = (occupation_result_copy['rank_tool_consumption_occ'] * weights['rank_tool_consumption_occ'] +
                          occupation_result_copy['rank_emp_occ_sum'] * weights['rank_emp_occ_sum'] +
                          occupation_result_copy['rank_unique_FIPS'] * weights['rank_unique_FIPS']
                          ) # Display the first few rows
occupation_sorted = occupation_result_copy.sort_values(by='Weighted_Sum_Occ', ascending=True)

In [ ]:
occupation_sorted.head(20)

,OCC_CODE,OCC_TITLE,tool_consumption_occ,emp_occ_sum,unique_FIPS,rank_tool_consumption_occ,rank_emp_occ_sum,rank_unique_FIPS,Weighted_Sum_Occ
33,51-4121,"Welders, Cutters, Solderers, and Brazers",10,31031.178369,1626,1,34,119,34.5
94,49-3042,"Mobile Heavy Equipment Mechanics, Except Engines",10,11320.601815,1623,1,95,121,53.2
97,49-9071,"Maintenance and Repair Workers, General",10,10820.961193,1625,1,98,120,53.9
9,17-2112,Industrial Engineers,8,64679.936227,1982,100,10,66,66.2
5,17-2141,Mechanical Engineers,8,89165.227078,1923,100,6,77,67.2
18,17-2011,Aerospace Engineers,8,38735.718806,1923,100,19,77,71.1
141,51-4041,Machinists,10,6871.857209,1547,1,142,146,72.3
57,49-9071,"Maintenance and Repair Workers, General",8,18815.469388,2271,100,58,38,75.0
166,47-5022,Excavating and Loading Machine and Dragline Op...,10,5441.470777,1583,1,167,129,76.4
40,51-4041,Machinists,8,25499.165442,1939,100,41,71,76.5


In [ ]:
top20_occ = occupation_sorted[:20]['OCC_CODE'].tolist()

In [ ]:
top20_occ

['51-4121',
 '49-3042',
 '49-9071',
 '17-2112',
 '17-2141',
 '17-2011',
 '51-4041',
 '49-9071',
 '47-5022',
 '51-4041',
 '49-9041',
 '49-3011',
 '11-3051',
 '47-2011',
 '17-3027',
 '11-3051',
 '51-4121',
 '17-3026',
 '49-3031',
 '51-2011']


### NOW, gonna filter the dataframe of the occupations and the patterna and merge all to have one master dataframe

In [ ]:
filtered_pattern_data = processed_df[processed_df['naics'].isin(top20_industries)]
filtered_occupation_data = df_filtered_occupation_main[df_filtered_occupation_main['OCC_CODE'].isin(top20_occ)]

In [ ]:
filtered_occupation_data

,FIPS,naics,NAICS_TITLE,OCC_CODE,OCC_TITLE,emp_occupation
0,12999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",20639.514235
1,6999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",9414.167765
2,36999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",8332.850279
4,48999,5613,Employment Services,49-9071,"Maintenance and Repair Workers, General",5770.378034
5,18039,3362,Motor Vehicle Body and Trailer Manufacturing,51-4121,"Welders, Cutters, Solderers, and Brazers",5303.207380
...,...,...,...,...,...,...
1020143,17143,4811,Scheduled Air Transportation,51-2011,"Aircraft Structure, Surfaces, Rigging, and Sys...",0.000000
1020144,17143,5613,Employment Services,51-2011,"Aircraft Structure, Surfaces, Rigging, and Sys...",0.000000
1020145,17141,5413,"Architectural, Engineering, and Related Services",51-2011,"Aircraft Structure, Surfaces, Rigging, and Sys...",0.000000
1020146,17137,5413,"Architectural, Engineering, and Related Services",51-2011,"Aircraft Structure, Surfaces, Rigging, and Sys...",0.000000


In [ ]:
# Merge Pattern and Occupation Data
merged_data = pd.merge(
    filtered_pattern_data,
    filtered_occupation_data,
    on='FIPS',
    how='inner'
)

# # Merge with GDP Data
# final_merged_data = pd.merge(
#     merged_data,
#     gdp_data[['FIPS', '2022']],  # Include only relevant columns from GDP data
#     on='FIPS',
#     how='inner'
# )


In [ ]:
merged_data

,FIPS,naics_x,DESCRIPTION,emp,ap,est,tool_consumption,naics_y,NAICS_TITLE,OCC_CODE,OCC_TITLE,emp_occupation
0,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,7,2123,Nonmetallic Mineral Mining and Quarrying,47-5022,Excavating and Loading Machine and Dragline Op...,9.359646
1,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,7,8111,Automotive Repair and Maintenance,49-3031,Bus and Truck Mechanics and Diesel Engine Spec...,7.276589
2,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,7,2211,"Electric Power Generation, Transmission and Di...",49-9041,Industrial Machinery Mechanics,6.767063
3,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,7,2123,Nonmetallic Mineral Mining and Quarrying,49-9041,Industrial Machinery Mechanics,5.650169
4,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,7,2123,Nonmetallic Mineral Mining and Quarrying,49-3042,"Mobile Heavy Equipment Mechanics, Except Engines",5.474887
...,...,...,...,...,...,...,...,...,...,...,...,...
5505898,56999,5613,Employment Services,22,676,13,5,2111,Oil and Gas Extraction,17-3027,Mechanical Engineering Technologists and Techn...,0.000000
5505899,56999,5613,Employment Services,22,676,13,5,2131,Support Activities for Mining,17-3027,Mechanical Engineering Technologists and Techn...,0.000000
5505900,56999,5613,Employment Services,22,676,13,5,2382,Building Equipment Contractors,17-3027,Mechanical Engineering Technologists and Techn...,0.000000
5505901,56999,5613,Employment Services,22,676,13,5,5413,"Architectural, Engineering, and Related Services",17-3027,Mechanical Engineering Technologists and Techn...,0.000000


In [ ]:
merge_columns_to_drop_gdp = [
    'tool_consumption', 'naics_y', 'NAICS_TITLE'
]

# Revomve not needed columns
filtered_merge = merged_data.drop(columns=merge_columns_to_drop_gdp, errors='ignore')

# Show the rest
print(filtered_merge.columns)

Index(['FIPS', 'naics_x', 'DESCRIPTION', 'emp', 'ap', 'est', 'OCC_CODE',
       'OCC_TITLE', 'emp_occupation'],
      dtype='object')


In [ ]:
filtered_merge

,FIPS,naics_x,DESCRIPTION,emp,ap,est,OCC_CODE,OCC_TITLE,emp_occupation
0,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,47-5022,Excavating and Loading Machine and Dragline Op...,9.359646
1,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,49-3031,Bus and Truck Mechanics and Diesel Engine Spec...,7.276589
2,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,49-9041,Industrial Machinery Mechanics,6.767063
3,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,49-9041,Industrial Machinery Mechanics,5.650169
4,1001,2211,"Electric Power Generation, Transmission and Di...",129,16342,4,49-3042,"Mobile Heavy Equipment Mechanics, Except Engines",5.474887
...,...,...,...,...,...,...,...,...,...
5505898,56999,5613,Employment Services,22,676,13,17-3027,Mechanical Engineering Technologists and Techn...,0.000000
5505899,56999,5613,Employment Services,22,676,13,17-3027,Mechanical Engineering Technologists and Techn...,0.000000
5505900,56999,5613,Employment Services,22,676,13,17-3027,Mechanical Engineering Technologists and Techn...,0.000000
5505901,56999,5613,Employment Services,22,676,13,17-3027,Mechanical Engineering Technologists and Techn...,0.000000


In [ ]:
# Merge with GDP Data
final_merged_data = pd.merge(
    filtered_merge,
    df_filtered_gdp[['FIPS', '2022']],  # Include only relevant columns from GDP data
    on='FIPS',
    how='inner'
)

KeyboardInterrupt: 

**Now, we will filter the Occupations for the ones that is gonna use our products**

*   Assume some occupations that are important like
  1.   '51-4121' > Welders, Cutters, Solderers, and Brazers
  2.   '47-2221' > Structural Iron and Steel Workers
  3.   '51-2041' > Structural Metal Fabricators and Fitters
  4.   '49-3021' > Automotive Body and Related Repairers
  5.   '51-4041' > Machinists
  6.   '49-9041' > Industrial Machinery Mechanics
  7.   '49-9071' > Maintenance and Repair Workers, General
  8.   '51-4081' > Multiple Machine Tool Setters, Operators, and Tenders, Metal and Plastic
  9.   '47-2211' > Sheet Metal Workers
  10.  '49-3031' > Bus and Truck Mechanics and Diesel Engine Specialists
  11.   '51-4033' > Grinding, Lapping, Polishing, and Buffing Machine Tool Setters, Operators, and Tenders, Metal and Plastic
  12.   '49-3023' > Automotive Service Technicians and Mechanics
  13.   '47-2011' > Boilermakers
  14.   '51-4122' > Welding, Soldering, and Brazing Machine Setters, Operators, and Tenders
  15.   '51-9021' > Crushing, Grinding, and Polishing Machine Setters, Operators, and Tenders
  16.   '51-4031' > Cutting, Punching, and Press Machine Setters, Operators, and Tenders, Metal and Plastic
  17.   '49-3011' > Aircraft Mechanics and Service Technicians
  18.   '51-4111' > Tool and Die Makers
  19.   '51-9032' > Cutting and Slicing Machine Setters, Operators, and Tenders
  20.   '49-9043' > Maintenance Workers, Machinery


# Feature overview

# Feature selection

# Feature construction

# feature_df

# Scaling

# Dimension reduction

# **Loading Map & Ploting it**

After looking in the County_Data

*   The column named "GEOID", should be remaned to "FIPS"
*   The FIPS column datatype should be changed to int


In [ ]:
county_data.rename(columns={'GEOID': 'FIPS'}, inplace=True)

In [ ]:
county_data = county_data.astype({"FIPS": int})

We will load the map csv amd plot it

Then apply then will merge it to the other dateframes

In [ ]:
# دمج البيانات الجغرافية مع الـ GDP
merged_data = pd.merge(county_data, gdp_data, on='GEOID', how='inner')

# دمج البيانات الصناعية
merged_data = pd.merge(merged_data, pattern_data, on='FIPS', how='inner')

# دمج بيانات الوظائف
merged_data = pd.merge(merged_data, occupation_data, on='FIPS', how='inner')

# دمج بيانات الولايات
merged_data = pd.merge(merged_data, state_data, on='STATEFP', how='inner')


NameError: name 'gdp_data' is not defined

## PCA

In [ ]:
# This code extracts the top 10 and the next 10 'OCC_CODE' values from the weighted ranked occupation data and stores them
# in two separate lists: 'occ_top10' and 'occ_top10_20'.
# It also defines two lists of NAICS codes: 'naics_top6' for the top 6 NAICS codes and 'naics_top_metall' for NAICS codes
# related to the metallurgical industry.
# # Finally, it prints the contents of the 'occ_top10', 'occ_top10_20', 'naics_top6', and 'naics_top_metall' lists.

occ_top10 = ranked_occupation_data_weighted['OCC_CODE'][:10].tolist()
occ_top20 = ['51-4121', '47-2221', '51-2041', '49-3021', '51-4041', '49-9041', '49-9071', '51-4081', '47-2211', '49-3031']
# #occ_top10_20 = ranked_occupation_data_weighted['OCC_CODE'][10:20].tolist()
# occ_top10_20 = ['51-4033', '49-3023', '47-2011', '51-4122', '51-9021', '51-4031', '49-3011', '51-4111', '51-9032', '49-9043']
# naics_top6 = ['2382','8111','3320A2','3330A1','3327','3363']
# naics_top_metall = ['3320A1','3335','3364','3362','3315','3366','2379','3336','3311','3314']
# print(occ_top10)
# print(occ_top10_20)
# print(naics_top6)
# print(naics_top_metall)